# String Constraints

Similarly to numerical constraints defined with the `Field` object, we also have constraints that are applicable to strings, and in fact, are applicable to sequence types in general, not just strings, even though the documentation does not seem to mention that.

These constraints are:
- `min_length` -> minimum length of string (or sequence)
- `max_length` -> maximum length of string (or sequence)

(Docs are located [here](https://docs.pydantic.dev/latest/api/fields/))

In [1]:
from pydantic import BaseModel, Field, ValidationError

In [2]:
class Model(BaseModel):
    name: str = Field(min_length=1, max_length=20)

In [3]:
Model(name="Richard")

Model(name='Richard')

In [4]:
try:
    Model(name="")
except ValidationError as ex:
    print(ex)

1 validation error for Model
name
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_short


In [5]:
try:
    Model(name="*" * 21)
except ValidationError as ex:
    print(ex)

1 validation error for Model
name
  String should have at most 20 characters [type=string_too_long, input_value='*********************', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_too_long


These constraints also apply to sequence types, such as lists and tuples:

In [6]:
class Model(BaseModel):
    items: list[float] = Field(min_length=3, max_length=5, default=[1.0, 2.0, 3.0])

In [7]:
Model()

Model(items=[1.0, 2.0, 3.0])

In [8]:
Model(items=[2, 3.5, 4])

Model(items=[2.0, 3.5, 4.0])

In [9]:
try:
    Model(items=[1, 2])
except ValidationError as ex:
    print(ex)

1 validation error for Model
items
  List should have at least 3 items after validation, not 2 [type=too_short, input_value=[1, 2], input_type=list]
    For further information visit https://errors.pydantic.dev/2.5/v/too_short


We can even use this approach (although we'll see shortly a slightly simpler way of doing this) to get back to an example we looked at previously. We wanted to define a `Circle` class, where the `center` had to be a 2 or 3 element tuple.

We ended up using `conlist`, but this meant having to setlle for a list type instead of a tuple.

We can use the above approach, as long as we can somehow specify a variadic tuple - e.g. a tuple where we can specify the type of the elements and the fact that there can be as many as we want (which we'll then restrict using `Field`).

So, this will not work, because the type hint we use indicates that the tuple can only contain a single integer element:

In [10]:
class Circle(BaseModel):
    center: tuple[int]
    radius: int = Field(gt=0, default=1)

In [11]:
try:
    Circle(center=(1, 1), radius=1)
except ValidationError as ex:
    print(ex)

1 validation error for Circle
center
  Tuple should have at most 1 item after validation, not 2 [type=too_long, input_value=(1, 1), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.5/v/too_long


As you can see, the way type hinting works with tuples is not quite the same as with lists - so we just need to modify our type hint to indicate the tuple has a variable length.

In [12]:
class Circle(BaseModel):
    center: tuple[int, ...]
    radius: int = Field(gt=0, default=1)

In [13]:
Circle(center=(1, 1))

Circle(center=(1, 1), radius=1)

In [14]:
Circle(center=(1, 1, 1))

Circle(center=(1, 1, 1), radius=1)

In [15]:
Circle(center=(1, 2, 3, 4, 5))

Circle(center=(1, 2, 3, 4, 5), radius=1)

That seems to work, and all we now need to do is constrain the tuple length:

In [16]:
class Circle(BaseModel):
    center: tuple[int, ...] = Field(min_length=2, max_length=3, default=(0, 0))
    radius: int = Field(gt=0, default=1)

In [17]:
Circle()

Circle(center=(0, 0), radius=1)

In [18]:
Circle(center=(1, 2, 3))

Circle(center=(1, 2, 3), radius=1)

In [19]:
try:
    Circle(center=(1, ))
except ValidationError as ex:
    print(ex)

1 validation error for Circle
center
  Value should have at least 2 items after validation, not 1 [type=too_short, input_value=(1,), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.5/v/too_short


In [20]:
try:
    Circle(center=(1, 2, 3, 4))
except ValidationError as ex:
    print(ex)

1 validation error for Circle
center
  Value should have at most 3 items after validation, not 4 [type=too_long, input_value=(1, 2, 3, 4), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.5/v/too_long


One additional feature in the `Field` class that is available, and for strings, not more general sequence types, is a constraint based on regular expressions.

I am not going to get into regex in this course (and I will not be making a course or YouTube videos on regex - I know the basics, but find anything else confusing).

However, if you are familiar with regex, let me show you how you can use it in string constraints.

We can specify a regex for a string constraint using the `pattern` argument in the `Field` class.

In [21]:
class Model(BaseModel):
    zip_code: str = Field(pattern=r"^[0-9]{5}(?:-[0-9]{4})?$")

In [22]:
Model(zip_code="12345")

Model(zip_code='12345')

In [23]:
Model(zip_code="12345-1234")

Model(zip_code='12345-1234')

In [24]:
try:
    Model(zip_code="12345-12345")
except ValidationError as ex:
    print(ex)

1 validation error for Model
zip_code
  String should match pattern '^[0-9]{5}(?:-[0-9]{4})?$' [type=string_pattern_mismatch, input_value='12345-12345', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/string_pattern_mismatch
